In [123]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import lightgbm as lgbm
from sklearn.ensemble import StackingClassifier


In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

2023-03-06 14:44:03.431118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 14:44:03.673315: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-06 14:44:03.673336: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-06 14:44:03.701899: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-06 14:44:04.507158: W tensorflow/stream_executor/platform/de

In [2]:
def get_performance(predictions, y_test, labels=[1, 0]):
    # Put your code
    accuracy = accuracy_score(y_test, predictions)
    precision, recall, f1_score, support = precision_recall_fscore_support(y_test, predictions)
    precision = precision[1]
    recall = recall[1]
    f1_score = f1_score[1]
    
    report = classification_report(y_test, predictions)
    
    cm = confusion_matrix(y_test, predictions)  # replace
    cm_as_dataframe = pd.DataFrame(data=cm)
    
    print('Model Performance metrics:')
    print('-'*30)
    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1 Score:', f1_score)
    print('\nModel Classification report:')
    print('-'*30)
    print(report)
    print('\nPrediction Confusion Matrix:')
    print('-'*30)
    print(cm_as_dataframe)
    
    return accuracy, precision, recall, f1_score

In [3]:
X_train=pd.read_csv('X_train.csv')
X_test=pd.read_csv('X_test.csv')
y_train=pd.read_csv('y_train.csv')
y_test=pd.read_csv('y_test.csv')

In [4]:
X_train=X_train.drop(columns=['Unnamed: 0'],axis=1)
X_test=X_test.drop(columns=['Unnamed: 0'],axis=1)

In [5]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.827377,0.737695,-0.104637,-0.474545,-0.473674,-0.581387
1,0.827377,-1.355574,-0.949986,3.154809,2.008933,-0.580634
2,0.827377,0.737695,0.049062,0.432793,-0.473674,-0.369661
3,0.827377,0.737695,-0.565736,-0.474545,-0.473674,-0.598054
4,-0.369365,-1.355574,1.201810,-0.474545,-0.473674,-0.436759
...,...,...,...,...,...,...
663,-1.566107,-1.355574,0.740711,0.432793,0.767630,1.360920
664,0.827377,-1.355574,-0.796286,0.432793,-0.473674,-0.582461
665,0.827377,0.737695,-0.104637,-0.474545,-0.473674,-0.585581
666,0.827377,-1.355574,0.510161,0.432793,-0.473674,-0.336112


In [6]:
y_train=y_train.drop(columns=['Unnamed: 0'],axis=1)
y_test=y_test.drop(columns=['Unnamed: 0'],axis=1)

#### Baseline

In [97]:
dtc=DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=50, min_samples_split=2, min_samples_leaf=1)

In [98]:
dtc.fit(X_train,y_train['Survived'])

DecisionTreeClassifier(max_depth=50)

In [99]:
y_test_pred=dtc.predict(X_test)

In [100]:
y_test_pred

array([0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 1])

In [101]:
y_test['Survived']

0      0
1      0
2      0
3      1
4      1
      ..
218    0
219    1
220    0
221    1
222    1
Name: Survived, Length: 223, dtype: int64

In [102]:
accuracy_score(y_test['Survived'],y_test_pred)

0.7623318385650224

In [103]:
sum(y_test['Survived']==y_test_pred)/len(y_test)

0.7623318385650224

In [104]:
confusion_matrix(y_test['Survived'],y_test_pred)

array([[114,  25],
       [ 28,  56]])

In [105]:
get_performance(y_test_pred, y_test['Survived'])

Model Performance metrics:
------------------------------
Accuracy: 0.7623318385650224
Precision: 0.691358024691358
Recall: 0.6666666666666666
F1 Score: 0.6787878787878788

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       139
           1       0.69      0.67      0.68        84

    accuracy                           0.76       223
   macro avg       0.75      0.74      0.75       223
weighted avg       0.76      0.76      0.76       223


Prediction Confusion Matrix:
------------------------------
     0   1
0  114  25
1   28  56


(0.7623318385650224, 0.691358024691358, 0.6666666666666666, 0.6787878787878788)

### Random Forest

In [124]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2,3,5],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 250]
}
rf = RandomForestClassifier()
grid_search_rf = GridSearchCV(rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)


In [125]:
grid_search_rf.fit(X_train,y_train['Survived'])

Fitting 3 folds for each of 324 candidates, totalling 972 fits
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=250; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110],
                         'max_features': [2, 3, 5],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 200, 250]},
             verbose=2)

In [127]:
grid_search_rf.best_estimator_

RandomForestClassifier(max_depth=90, max_features=2, min_samples_leaf=3,
                       min_samples_split=12, n_estimators=200)

In [128]:
y_test_pred_rf=grid_search_rf.predict(X_test)

In [129]:
get_performance(y_test_pred_rf, y_test['Survived'])

Model Performance metrics:
------------------------------
Accuracy: 0.8430493273542601
Precision: 0.8450704225352113
Recall: 0.7142857142857143
F1 Score: 0.7741935483870968

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       139
           1       0.85      0.71      0.77        84

    accuracy                           0.84       223
   macro avg       0.84      0.82      0.83       223
weighted avg       0.84      0.84      0.84       223


Prediction Confusion Matrix:
------------------------------
     0   1
0  128  11
1   24  60


(0.8430493273542601,
 0.8450704225352113,
 0.7142857142857143,
 0.7741935483870968)

### XGboost

In [130]:

param_dist = {
    'n_estimators':[5,6,7,8],
    'max_depth':[30,40,50], 
    'learning_rate':[0.2,0.25,0.3],
    }
#Create the lgbm moodel object
bst = XGBClassifier(objective='binary:logistic')
#Create the GridSearch object
bst_grid = GridSearchCV(bst, param_dist,cv = 5, n_jobs = -1, verbose = 2)

In [131]:

bst_grid.fit(X_train,y_train['Survived'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=6; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=6; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=6; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=6; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=7; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=6; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n_estimators=7; total time=   0.0s
[CV] END ....learning_rate=0.2, max_depth=30, n

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.2, 0.25, 0.3],
                         'max_depth': [30, 40, 50],
                         'n_estimators': [5, 6, 7, 8]},
             verbose=2)

In [132]:
bst_grid.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.25, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=30, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=6, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [133]:
y_test_pred_lgbm=bst_grid.predict(X_test)
get_performance(y_test_pred_lgbm, y_test['Survived'])

Model Performance metrics:
------------------------------
Accuracy: 0.8609865470852018
Precision: 0.8840579710144928
Recall: 0.7261904761904762
F1 Score: 0.7973856209150327

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.85      0.94      0.89       139
           1       0.88      0.73      0.80        84

    accuracy                           0.86       223
   macro avg       0.87      0.83      0.85       223
weighted avg       0.86      0.86      0.86       223


Prediction Confusion Matrix:
------------------------------
     0   1
0  131   8
1   23  61


(0.8609865470852018,
 0.8840579710144928,
 0.7261904761904762,
 0.7973856209150327)

#### LightGBM

In [119]:
param_dist_lgbm = {
         'num_iterations': [7000,8000],
     'learning_rate':[  0.001,0.0008,0.0005 ],
    'num_leaves':[ 31,45 ],
    }

param_dist_lgbm_2 = {
    'learning_rate' : [0.01,0.008,0.001],
'num_leaves' : [50,100,150],
'num_trees' : [300,350,400],
}
#Create the lgbm moodel object
lgbmClass = lgbm.LGBMClassifier(boosting_type='gbdt',  objective='binary',metric='rmse',tree_learner='voting')
#Create the GridSearch object
lgbm_grid = GridSearchCV(lgbmClass, param_dist_lgbm,cv = 5, n_jobs = -1)

In [120]:
lgbm_grid.fit(X_train,y_train['Survived'])

/home/manuelquiros/Documents/VENVS/KaggleTitanic/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/manuelquiros/Documents/VENVS/KaggleTitanic/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/manuelquiros/Documents/VENVS/KaggleTitanic/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/manuelquiros/Documents/VENVS/KaggleTitanic/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params

GridSearchCV(cv=5,
             estimator=LGBMClassifier(metric='rmse', objective='binary',
                                      tree_learner='voting'),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.0008, 0.0005],
                         'num_iterations': [7000, 8000],
                         'num_leaves': [31, 45]})

In [121]:
lgbm_grid.best_estimator_

LGBMClassifier(learning_rate=0.0008, metric='rmse', num_iterations=7000,
               objective='binary', tree_learner='voting')

In [122]:
y_test_pred_lgbm=lgbm_grid.predict(X_test)
get_performance(y_test_pred_lgbm, y_test['Survived'])

Model Performance metrics:
------------------------------
Accuracy: 0.8609865470852018
Precision: 0.863013698630137
Recall: 0.75
F1 Score: 0.802547770700637

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.86      0.93      0.89       139
           1       0.86      0.75      0.80        84

    accuracy                           0.86       223
   macro avg       0.86      0.84      0.85       223
weighted avg       0.86      0.86      0.86       223


Prediction Confusion Matrix:
------------------------------
     0   1
0  129  10
1   21  63


(0.8609865470852018, 0.863013698630137, 0.75, 0.802547770700637)

#### Stacking model

In [136]:
from sklearn.linear_model import LogisticRegression

In [139]:
estimators = [
('rf', grid_search_rf.best_estimator_),
('xgboost', bst_grid.best_estimator_),
('lgbm',lgbm_grid.best_estimator_)]

In [146]:
param_dist = {
         'tol' : [0.0001, 0.0005, 0.00005, 0.001],
'C' :  [0.001, 0.01, 0.1, 1],
'max_iter' : [250, 300, 350],
    }


#Create the lgbm moodel object
logreg = LogisticRegression()
#Create the GridSearch object
logreg_grid = GridSearchCV(logreg, param_dist,cv = 5, n_jobs = -1)

In [150]:
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(tol=0.001,C=0.1,max_iter=300))

In [151]:
clf.fit(X_train,y_train['Survived'])

/home/manuelquiros/Documents/VENVS/KaggleTitanic/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/manuelquiros/Documents/VENVS/KaggleTitanic/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/manuelquiros/Documents/VENVS/KaggleTitanic/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/manuelquiros/Documents/VENVS/KaggleTitanic/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=90,
                                                       max_features=2,
                                                       min_samples_leaf=3,
                                                       min_samples_split=12,
                                                       n_estimators=200)),
                               ('xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=Non...
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=6, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...)),
                               ('lgbm',
                                LGBMClassifier(learning_rate=0.0008,
                                               metric='rmse',
                                               num_iterations=7000,
                                               objective='binary',
                                               tree_learner='voting'))],
                   final_estimator=LogisticRegression(C=0.1, max_iter=300,
                                                      tol=0.001))

In [152]:
y_test_pred_stack=clf.predict(X_test)
get_performance(y_test_pred_stack, y_test['Survived'])

Model Performance metrics:
------------------------------
Accuracy: 0.852017937219731
Precision: 0.8695652173913043
Recall: 0.7142857142857143
F1 Score: 0.7843137254901961

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       139
           1       0.87      0.71      0.78        84

    accuracy                           0.85       223
   macro avg       0.86      0.82      0.84       223
weighted avg       0.85      0.85      0.85       223


Prediction Confusion Matrix:
------------------------------
     0   1
0  130   9
1   24  60


(0.852017937219731, 0.8695652173913043, 0.7142857142857143, 0.7843137254901961)

### NN

In [8]:
model = keras.Sequential()
model.add(layers.Input(shape=6))
#model.add(layers.Dense(8, input_dim=6, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               896       
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 512)               131584    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 2)                

In [14]:
opt = keras.optimizers.Adam(learning_rate=0.0007)

In [15]:
model.compile(
  loss='sparse_categorical_crossentropy', 
  optimizer=opt,
  metrics=['accuracy'],
)

In [16]:
cp2 = keras.callbacks.ModelCheckpoint('/home/manuelquiros/Documents/Kaggle/Titanic/MLP4/', save_best_only=True)
log_dir2 = "MLP4/logs/fit/"
tb2 = keras.callbacks.TensorBoard(log_dir=log_dir2, histogram_freq=1)
es2 = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30)
lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,patience=15, min_lr=0.0000001)

In [17]:
history = model.fit(
  X_train, 
  y_train['Survived'], 
  epochs=200, 
  batch_size=8, 
  validation_split=0.2,
  callbacks=[cp2,tb2,lr]
)

Epoch 1/200
67/67 [==============================] - 1s 13ms/step - loss: 0.3431 - accuracy: 0.8558 - val_loss: 0.6966 - val_accuracy: 0.8134 - lr: 7.0000e-04
Epoch 2/200
67/67 [==============================] - 0s 3ms/step - loss: 0.3328 - accuracy: 0.8596 - val_loss: 0.7287 - val_accuracy: 0.7761 - lr: 7.0000e-04
Epoch 3/200
67/67 [==============================] - 0s 3ms/step - loss: 0.3261 - accuracy: 0.8558 - val_loss: 0.7417 - val_accuracy: 0.7985 - lr: 7.0000e-04
Epoch 4/200
67/67 [==============================] - 0s 3ms/step - loss: 0.3144 - accuracy: 0.8652 - val_loss: 0.9033 - val_accuracy: 0.8134 - lr: 7.0000e-04
Epoch 5/200
67/67 [==============================] - 0s 3ms/step - loss: 0.3090 - accuracy: 0.8689 - val_loss: 0.7411 - val_accuracy: 0.8134 - lr: 7.0000e-04
Epoch 6/200
67/67 [==============================] - 1s 10ms/step - loss: 0.3285 - accuracy: 0.8633 - val_loss: 0.6893 - val_accuracy: 0.8060 - lr: 7.0000e-04
Epoch 7/200
67/67 [==============================]

In [57]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.


In [18]:
model = keras.models.load_model('MLP4/')

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               896       
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 512)               131584    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 2)                

In [20]:
y_test_pred_mlp=model.predict(X_test)

7/7 [==============================] - 0s 1ms/step


In [21]:
y_test_pred_mlp

array([[6.49558246e-01, 3.45523208e-01],
       [6.53905571e-01, 3.41466665e-01],
       [9.94224489e-01, 1.27259607e-03],
       [3.40912928e-04, 9.99997258e-01],
       [4.71421927e-01, 5.54072857e-01],
       [4.20280397e-01, 6.29346311e-01],
       [3.02645145e-04, 9.99997795e-01],
       [8.71870637e-01, 9.61409956e-02],
       [9.36151326e-01, 3.94508392e-02],
       [4.71381724e-01, 5.51895738e-01],
       [9.22008038e-01, 4.46306951e-02],
       [2.89616019e-01, 8.14969420e-01],
       [6.53787792e-01, 3.41572404e-01],
       [2.22888077e-04, 9.99998629e-01],
       [2.08264813e-02, 9.98056710e-01],
       [4.41594392e-01, 6.01157188e-01],
       [6.92442954e-01, 3.03713441e-01],
       [8.71134363e-03, 9.99441147e-01],
       [6.51527047e-01, 3.42663467e-01],
       [1.01857149e-04, 9.99999523e-01],
       [6.74055994e-01, 3.25337946e-01],
       [1.12599349e-02, 9.99259591e-01],
       [6.53796077e-01, 3.41564804e-01],
       [4.31709975e-01, 6.13096833e-01],
       [4.538016

In [314]:
y_test_pred_mlp = [int(item[1]) for item in y_test_pred_mlp.round()]

In [315]:
y_test_pred_mlp

[0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1]

In [306]:
loss, accuracy = model.evaluate(X_test, y_test['Survived'])

7/7 [==============================] - 0s 739us/step - loss: 0.3882 - accuracy: 0.8206


### Validation predictions

In [22]:
#change the model in the predict line
X_val = pd.read_csv('X_val.csv')
X_val_pred = X_val.drop(columns=['Unnamed: 0','PassengerId'],axis=1)
y_val_pred=model.predict(X_val_pred)
#run only if done with MLP
y_val_pred = [int(item[1]) for item in y_val_pred.round()]
X_val['Survived']=y_val_pred
output=X_val.drop(columns=['Unnamed: 0',  'Pclass', 'Sex', 'Age', 'SibSp', 'Parch','Fare'],axis=1)
output.to_csv('output.csv',index=False)

14/14 [==============================] - 0s 755us/step
